In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append('..')

In [64]:
import inspect
from io import StringIO

from epypes import dag

In [3]:
g = dag.Digraph(range(10), [(0, 1), (5, 4), (5, 6)])

In [4]:
g.reverse()._adj

{0: [], 1: [0], 2: [], 3: [], 4: [5], 5: [], 6: [5], 7: [], 8: [], 9: []}

In [5]:
def plus(a, b):
    return a + b

def triple(x):
    return 3 * x

cg = dag.ComputationalGraph([plus, triple], ['a', 'b', 's', 'res'])
cg.add_edge('a', 'plus')
cg.add_edge('b', 'plus')
cg.add_edge('plus', 's')
cg.add_edge('s', 'triple')
cg.add_edge('triple', 'res')

cg._adj

{'a': ['plus'],
 'b': ['plus'],
 'plus': ['s'],
 'res': [],
 's': ['triple'],
 'triple': ['res']}

In [6]:
cg._functions

{'plus': <function __main__.plus>, 'triple': <function __main__.triple>}

In [27]:
content_tinyDAG = '''
13
15
2 3 
0 6 
0 1 
2 0 
11 12  
9 12  
9 10  
9 11 
3 5 
8 7 
5 4 
0 5 
6 4 
6 9 
7 6
'''

In [29]:
content_tinyDG = '''
13
22
 4  2
 2  3
 3  2
 6  0
 0  1
 2  0
11 12
12  9
 9 10
 9 11
 7  9
10 12
11  4
 4  3
 3  5
 6  8
 8  6
 5  4
 0  5
 6  4
 6  9
 7  6
'''

In [74]:
from epypes import dag

def read_graph(buf):
    buf.readline()
    num_vertices = int(buf.readline().strip())
    num_edges = int(buf.readline().strip())
    
    edges = []
    for pair in buf:
        a, b = pair.strip().split()
        edges.append((int(a), int(b)))
    
    return num_vertices, num_edges, edges
        
        
buf = StringIO(content_tinyDG)
num_vertices, num_edges, edges = read_graph(buf)

tinyDG = dag.Digraph(range(num_vertices), edges)

dc = dag.DirectedCycle(tinyDG)

0 1 [0]
0 5 [0]
5 4 [0, 5]
4 2 [0, 4, 5]
2 3 [0, 2, 4, 5]
3 2 [0, 2, 3, 4, 5]
2 is on stack!
[3, 2, 3]
3 5 [0, 2, 3, 4, 5]
2 0 [0, 2, 3, 4, 5]
4 3 [0, 2, 3, 4, 5]
6 0 [2, 3, 4, 6]
7 9 [2, 3, 4, 6, 7]
8 6 [2, 3, 4, 6, 7, 8]
9 10 [2, 3, 4, 6, 7, 8, 9]
10 12 [2, 3, 4, 6, 7, 8, 9, 10]
11 12 [2, 3, 4, 6, 7, 8, 9, 10, 11]
12 9 [2, 3, 4, 6, 7, 8, 9, 10, 11, 12]


In [82]:
from epypes import dag
        
num_vertices, num_edges, edges = read_graph(StringIO(content_tinyDAG))

tinyDAG = dag.Digraph(range(num_vertices), edges)

dc = dag.DirectedCycle(tinyDAG)
dfo = dag.DepthFirstOrder(tinyDAG)

In [87]:
for v in dfo.reverse_postorder:
    print(v, tinyDAG.adj(v))

8 [7]
7 [6]
2 [3, 0]
3 [5]
0 [6, 1, 5]
5 [4]
1 []
6 [4, 9]
9 [12, 10, 11]
11 [12]
10 []
12 []
4 []
